# Installing and Getting Started with Ag2

## Installation

AG2 is available on [PyPI](https://pypi.org/project/ag2/) and can be installed using `pip`, the Python package manager.

**Basic installation (with OpenAI support):**

```bash
pip install "ag2[openai]"
```

**Install with additional model providers:**

```bash
pip install "ag2[anthropic,cohere,mistral,gemini]"
```

You can specify any combination of providers in the brackets, depending on your needs.

> **Tip:** We recommend using a [virtual environment](https://docs.python.org/3/tutorial/venv.html) (such as `venv` or `conda`) for your project to keep dependencies isolated and manageable.


In [ ]:
! pip install "ag2[openai]"

## Basic concepts

 ### LLM Configuration
 
 The **LLM Configuration** specifies the language model intelligence that powers your agents. It is the first component you should set up when building with AG2, as it determines how your agents will think, reason, and generate responses.
 
 The LLM Configuration allows you to:
 
 - **Connect to and authenticate** with language model providers
 - **Select models** and adjust their parameters
 - **Control how your agent thinks, reasons, and responds**
 
 Properly configuring your LLM ensures your agents have the right capabilities for your use case.

creating and LLM Configuration
we have 2 methods 
1) Using Direct Parameters
2) Using the config_list Parameter

Method 1: Using Direct Parameters

In [ ]:
import os
from autogen import LLMConfig

llm_config = LLMConfig(
    api_type="openai",                      # The provider
    model="gpt-5-nano",                    # The specific model
    api_key=os.environ["OPENAI_API_KEY"],   # Authentication
)

Method 2: Using the config_list Parameter
- For more advanced scenarios, especially when you want to set up fallback models, use the config_list parameter.



In [ ]:
import os
from autogen import LLMConfig

llm_config = LLMConfig(
    config_list=[
        {
            "api_type": "openai",
            "model": "gpt-5-nano",
            "api_key": os.environ["OPENAI_API_KEY"]
        },
        {
            "api_type": "openai",
            "model": "o3-mini",
            "api_key": os.environ["OPENAI_API_KEY"]
        }
    ],
)

### ConversableAgent

The ConversableAgent is the core building block of AG2 — a smart, interactive agent that uses your configured LLM to process information and interact with other agents or humans. With a properly configured LLM, your agents can:

- Communicate with other agents and humans
- Process information using Large Language Models (LLMs)
- Make decisions based on its defined purpose
- Execute tools and functions when needed

Every agent in your AG2 system is either a ConversableAgent or built upon one, making it the most important class to understand.



 ## Creating a `ConversableAgent`

 The `ConversableAgent` is the fundamental building block for building intelligent, interactive agents in AG2.

 **Key Parameters:**

 When initializing a `ConversableAgent`, consider the following important parameters:

 - **`name`**:  
   A unique identifier for your agent. This helps distinguish between multiple agents in your system.

 - **`system_message`**:  
   Instructions that define the agent's role, personality, and behavior. This message guides how the agent responds and interacts.

 - **`llm_config`**:  
   Configuration for the language model. This can be provided directly or via a context manager, and determines which LLM the agent uses.

In [ ]:
from autogen import ConversableAgent, LLMConfig

# Create LLM configuration first
llm_config = LLMConfig(api_type="openai", model="gpt-4o-mini")

# Create the agent using the context manager approach
my_agent = ConversableAgent(
    name="helpful_agent",  # Give your agent a unique name
    system_message="You are a helpful AI assistant",  # Define its personality and purpose
    llm_config=llm_config  # Pass the LLM configuration
)

**Interacting with a ConversableAgent**
- The simplest way to interact with a ConversableAgent is to use the run() and process() methods. Here's a basic example:

 ### Why Two Steps? `run()` and `process()`
 
 When you call `run()`, it does **not** immediately return the final output. Instead, it gives you an **iterator**—a special object that streams events, messages, and metadata as the conversation unfolds.
 
 The `process()` method is a convenient helper that automatically iterates through these events for you. It simulates a chat-like console experience: printing messages, handling user input, and making the interaction feel live and conversational.
 
 **Summary:**
 
 - Use **`run()`** and manually iterate over the events if you want **full control** over the workflow and how each event is handled.
 - Use **`process()`** (typically in combination with `run()`) for a **quick, ready-to-use chat experience** in the console.

In [ ]:
# Establish the workflow
response = my_agent.run(
    message="What's the capital of France?",
    max_turns=2,  # Limit conversation length
    user_input=True  # Allow user to provide input
)

# Process the workflow
response.process()

### 🏦 Financial Compliance Example

Let's build a simple **financial agent** using `ConversableAgent` to help analyze transactions and discuss compliance topics:

In [ ]:
from autogen import ConversableAgent, LLMConfig
import os

# Configure the LLM (we created this in the previous section)
llm_config = LLMConfig(
    api_type="openai",
    model="gpt-5-nano",
    api_key=os.environ["OPENAI_API_KEY"],
    temperature=0.2
)

# Create a basic financial agent
with llm_config:
    finance_agent = ConversableAgent(
        name="finance_agent",
        system_message="You are a financial assistant who helps analyze financial data and transactions."
    )

# Run the agent with a prompt
response = finance_agent.run(
    message="Can you explain what makes a transaction suspicious?",
    max_turns=1
)

# Iterate through the chat automatically with console output
response.process()

### Human in the Loop

Human in the Loop (HITL) is a powerful pattern that enables your AG2 agents to collaborate with humans during their workflow. Instead of making all decisions independently, agents can check with human operators at critical decision points, combining AI efficiency with human judgment.


- Enables human approval before proceeding in a workflow
- Integrates feedback into the decision-making process
- Balances automation with human judgment

 ## 🕵️‍♂️ When Should You Use Human in the Loop (HITL)?

 Human in the Loop (HITL) is especially important in scenarios where:

 - **Nuanced judgment is required:**  
   For example, in financial compliance or legal matters where context and expertise are crucial.
 - **Mistakes could have serious consequences:**  
   Such as in financial transactions or safety-critical systems, where errors can be costly or dangerous.
 - **Subjective input improves outcomes:**  
   Like content approval or design choices, where human taste and preferences matter.
 - **Regulations require human oversight:**  
   In industries such as financial services or healthcare, where laws mandate a human review step.


```
from autogen import ConversableAgent

# Create a human agent that will always prompt for input
human = ConversableAgent(
    name="human",
    human_input_mode="ALWAYS",  # Always ask for human input
)

# Create an AI agent that never asks for human input directly
ai_agent = ConversableAgent(
    name="ai_assistant",
    system_message="You are a helpful AI assistant",
    human_input_mode="NEVER",  # Never ask for human input directly
)
```

The human_input_mode parameter has three possible values:

1) ALWAYS: The agent uses the human input as its response
2) TERMINATE: The agent asks for input only when terminating a conversation
3) NEVER: The agent never asks for human input

**let's continue the financial complience example with (HITL)**

In [ ]:
from autogen import ConversableAgent, LLMConfig
import os
import random

# Note: Make sure to set your API key in your environment first

# Configure the LLM
llm_config = LLMConfig(
    api_type="openai",
    model="gpt-4o-mini",
    api_key=os.environ.get("OPENAI_API_KEY"),
    temperature=0.2,
)

# Define the system message for our finance bot
finance_system_message = """
You are a financial compliance assistant. You will be given a set of transaction descriptions.
For each transaction:
- If it seems suspicious (e.g., amount > $10,000, vendor is unusual, memo is vague), ask the human agent for approval.
- Otherwise, approve it automatically.
Provide the full set of transactions to approve at one time.
If the human gives a general approval, it applies to all transactions requiring approval.
When all transactions are processed, summarize the results and say "You can type exit to finish".
"""

# Create the finance agent with LLM intelligence
with llm_config:
    finance_bot = ConversableAgent(
        name="finance_bot",
        system_message=finance_system_message,
    )

# Create the human agent for oversight
human = ConversableAgent(
    name="human",
    human_input_mode="ALWAYS",  # Always ask for human input
)

# Generate sample transactions - this creates different transactions each time you run
VENDORS = ["Staples", "Acme Corp", "CyberSins Ltd", "Initech", "Globex", "Unicorn LLC"]
MEMOS = ["Quarterly supplies", "Confidential", "NDA services", "Routine payment", "Urgent request", "Reimbursement"]

def generate_transaction():
    amount = random.choice([500, 1500, 9999, 12000, 23000, 4000])
    vendor = random.choice(VENDORS)
    memo = random.choice(MEMOS)
    return f"Transaction: ${amount} to {vendor}. Memo: {memo}."

# Generate 3 random transactions
transactions = [generate_transaction() for _ in range(3)]

# Format the initial message
initial_prompt = (
    "Please process the following transactions one at a time:\n\n" +
    "\n".join([f"{i+1}. {tx}" for i, tx in enumerate(transactions)])
)

# Start the conversation from the human agent
response = human.run(
    recipient=finance_bot,
    message=initial_prompt,
)

# Display the response
response.process()

### Agent Orchestration

Agent Orchestration defines patterns for coordinating multiple agents, allowing them to work together in various configurations:

- Two-agent conversations
- Sequential conversations that chain multiple dialogues
- Group collaborations with many agents
- Nested workflows

AG2 offers several orchestration patterns, and for our evolving Agentic system, the Group Chat pattern is particularly powerful. It allows specialized agents to collaborate with dynamic handoffs to achieve complex workflows.



Here's how you implement a basic group chat:



In [ ]:
from autogen import ConversableAgent
from autogen.agentchat import initiate_group_chat
from autogen.agentchat.group.patterns import AutoPattern

# Create your specialized agents
with llm_config:
    agent_1 = ConversableAgent(name="agent_1", system_message="...")
    agent_2 = ConversableAgent(name="agent_2", system_message="...")

# Create human agent if needed
human = ConversableAgent(name="human", human_input_mode="ALWAYS")

# Set up the pattern for orchestration
pattern = AutoPattern(
    initial_agent=agent_1,              # Agent that starts the workflow
    agents=[agent_1, agent_2],          # All agents in the group chat
    user_agent=human,                   # Human agent for interaction
    group_manager_args={"llm_config": llm_config}  # Config for group manager
)

# Initialize the group chat
result, context_variables, last_agent = initiate_group_chat(
    pattern=pattern,
    messages="Initial request",         # Starting message
)

### Tools

The tools extend an agent’s capabilities beyond text conversations, enabling them to:

- Connect with external APIs and services
- Perform calculations and data processing
- Access and work with files, databases, or other systems

### how tool works
- Selection: An agent (driven by its LLM) decides which tool is appropriate based on the given task
- Execution: A separate executor agent invokes the tool and returns the results

In [ ]:
import os
import random
from typing import Annotated, Any
from datetime import datetime, timedelta

from autogen import ConversableAgent, LLMConfig
from autogen.agentchat import initiate_group_chat
from autogen.agentchat.group.patterns import AutoPattern

# Note: Make sure to set your API key in your environment first

# Configure the LLM
llm_config = LLMConfig(
    api_type="openai",
    model="gpt-4o-mini",
    api_key=os.environ.get("OPENAI_API_KEY"),
    temperature=0.2,
)

# Mock database of previous transactions
def get_previous_transactions() -> list[dict[str, Any]]:
    today = datetime.now()
    return [
        {
            "vendor": "Staples",
            "amount": 500,
            "date": (today - timedelta(days=3)).strftime("%Y-%m-%d"),  # 3 days ago
            "memo": "Quarterly supplies",
        },
        {
            "vendor": "Acme Corp",
            "amount": 1500,
            "date": (today - timedelta(days=10)).strftime("%Y-%m-%d"),  # 10 days ago
            "memo": "NDA services",
        },
        {
            "vendor": "Globex",
            "amount": 12000,
            "date": (today - timedelta(days=5)).strftime("%Y-%m-%d"),  # 5 days ago
            "memo": "Confidential",
        },
    ]

# Simple duplicate detection function
def check_duplicate_payment(
    vendor: Annotated[str, "The vendor name"],
    amount: Annotated[float, "The transaction amount"],
    memo: Annotated[str, "The transaction memo"]
) -> dict[str, Any]:
    """Check if a transaction appears to be a duplicate of a recent payment"""
    previous_transactions = get_previous_transactions()

    today = datetime.now()

    for tx in previous_transactions:
        tx_date = datetime.strptime(tx["date"], "%Y-%m-%d")
        date_diff = (today - tx_date).days

        # If vendor, memo and amount match, and transaction is within 7 days
        if (
            tx["vendor"] == vendor and
            tx["memo"] == memo and
            tx["amount"] == amount and
            date_diff <= 7
        ):
            return {
                "is_duplicate": True,
                "reason": f"Duplicate payment to {vendor} for ${amount} on {tx['date']}"
            }

    return {
        "is_duplicate": False,
        "reason": "No recent duplicates found"
    }

# Define the system message for our finance bot
finance_system_message = """
You are a financial compliance assistant. You will be given a set of transaction descriptions.

For each transaction:
1. First, extract the vendor name, amount, and memo
2. Check if the transaction is a duplicate using the check_duplicate_payment tool
3. If the tool identifies a duplicate, automatically reject the transaction
4. If not a duplicate, continue with normal evaluation:
    - If it seems suspicious (e.g., amount > $10,000, vendor is unusual, memo is vague), ask the human agent for approval
    - Otherwise, approve it automatically

Provide clear explanations for your decisions, especially for duplicates or suspicious transactions.
When all transactions are processed, summarize the results and say "You can type exit to finish".
"""

# Define the system message for the summary agent
summary_system_message = """
You are a financial summary assistant. You will be given a set of transaction details and their approval status.
Your task is to summarize the results of the transactions processed by the finance bot.
Generate a markdown table with the following columns:
- Vendor
- Memo
- Amount
- Status (Approved/Rejected)
- Reason (especially note if rejected due to being a duplicate)

The summary should include the total number of transactions, the number of approved transactions, and the number of rejected transactions.
The summary should be concise and clear.

Once you've generated the summary append the below in the summary:
==== SUMMARY GENERATED ====
"""

# Create the finance agent with LLM intelligence
with llm_config:
    finance_bot = ConversableAgent(
        name="finance_bot",
        system_message=finance_system_message,
        functions=[check_duplicate_payment],
    )
    summary_bot = ConversableAgent(
        name="summary_bot",
        system_message=summary_system_message,
    )

# Create the human agent for oversight
human = ConversableAgent(
    name="human",
    human_input_mode="ALWAYS",  # Always ask for human input
)

def is_termination_msg(msg: dict[str, Any]) -> bool:
    content = msg.get("content", "")
    return (content is not None) and "==== SUMMARY GENERATED ====" in content

# Generate sample transactions - this creates different transactions each time you run
VENDORS = ["Staples", "Acme Corp", "CyberSins Ltd", "Initech", "Globex", "Unicorn LLC"]
MEMOS = ["Quarterly supplies", "Confidential", "NDA services", "Routine payment", "Urgent request", "Reimbursement"]

# Generate new transactions including a duplicate
transactions = [
    "Transaction: $500 to Staples. Memo: Quarterly supplies.",  # Duplicate of an existing transaction
    "Transaction: $4000 to Unicorn LLC. Memo: Reimbursement.",
    "Transaction: $12000 to Globex. Memo: Confidential.",  # Duplicate of an existing transaction
    "Transaction: $22000 to Initech. Memo: Urgent request."
]

# Format the initial message
initial_prompt = (
    "Please process the following transactions one at a time, checking for duplicates:\n\n" +
    "\n".join([f"{i+1}. {tx}" for i, tx in enumerate(transactions)])
)

# Create pattern and start group chat
pattern = AutoPattern(
    initial_agent=finance_bot,
    agents=[finance_bot, summary_bot],
    user_agent=human,
    group_manager_args = {
        "llm_config": llm_config,
        "is_termination_msg": is_termination_msg
    },
)

result, _, _ = initiate_group_chat(
    pattern=pattern,
    messages=initial_prompt,
)

### Structured Outputs

Structured Outputs ensure agents return well-defined, consistent, and validated responses using Pydantic models. This allows you to:

- Define structured response formats
- Guarantee consistent data structures
- Simplify downstream processing and application integration
- Ensure responses are complete and reliable

**Implementing structured outputs in AG2 is straightforward using Pydantic models and the response_format parameter**

In [ ]:
from pydantic import BaseModel
from autogen import ConversableAgent, LLMConfig

# 1. Define your structured output model with Pydantic
class ResponseModel(BaseModel):
    field1: str
    field2: int
    field3: list[str]

# 2. Create LLM configuration with the structured output model
llm_config = LLMConfig(
    api_type="openai",
    model="gpt-4o-mini",
    response_format=ResponseModel,  # Specify the response format
)

# 3. Create agent with structured output configuration
with llm_config:
    structured_agent = ConversableAgent(
        name="structured_agent",
        system_message="You provide information in a structured format."
    )